In [2]:
# Import some useful modules.
import jax
import jax.numpy as np
import os
import glob

In [3]:
# Import JAX-FEM specific modules.
from jax_fem.generate_mesh import box_mesh_gmsh, Mesh, get_meshio_cell_type
from jax_fem.solver import solver
from jax_fem.problem import Problem
from jax_fem.utils import save_sol

       __       ___      ___   ___                _______  _______ .___  ___. 
      |  |     /   \     \  \ /  /               |   ____||   ____||   \/   | 
      |  |    /  ^  \     \  V  /      ______    |  |__   |  |__   |  \  /  | 
.--.  |  |   /  /_\  \     >   <      |______|   |   __|  |   __|  |  |\/|  | 
|  `--'  |  /  _____  \   /  .  \                |  |     |  |____ |  |  |  | 
 \______/  /__/     \__\ /__/ \__\               |__|     |_______||__|  |__| 
                                                                              



# Workbench for thermal conduction

---

In [23]:
# If you have multiple GPUs, set the one to use.
device = ' cpu'

if device == 'gpu':
    os.environ["CUDA_VISIBLE_DEVICES"] = "2"

# Define some useful directory paths.
# crt_file_path = os.path.dirname(__file__)
crt_file_path = os.getcwd()
data_dir = os.path.join(crt_file_path, '05a_data')
vtk_dir = os.path.join(data_dir, 'vtk')

In [24]:
# Define the thermal problem. 
# We solve the following equation (weak form of FEM):
# (rho*Cp/dt*(T_crt-T_old), Q) * dx + (k*T_crt_grad, Q_grad) * dx - (heat_flux, Q) * ds = 0
# where T_crt is the trial function, and Q is the test function.
class Thermal(Problem):
    # The function 'get_tensor_map' is responsible for the term (k*T_crt_grad, Q_grad) * dx. 
    # The function 'get_mass_map' is responsible for the term (rho*Cp/dt*(T_crt-T_old), Q) * dx. 
    # The function 'set_params' makes sure that the Neumann boundary conditions use the most 
    # updated T_old and laser information, i.e., positional information like laser_center (x_l, y_l) and 
    # 'switch' controlling ON/OFF of the laser.
    def get_tensor_map(self):
        def fn(u_grad, T_old):
            return k * u_grad
        return fn
 
    def get_mass_map(self):
        def T_map(T, x, T_old):
            return rho * Cp * (T - T_old) / dt
        return T_map

    def get_surface_maps(self):
        # Neumann BC values for thermal problem
        def thermal_neumann_top(u, point, old_T, laser_center, switch):
            # q is the heat flux into the domain
            d2 = (point[0] - laser_center[0]) ** 2 + (point[1] - laser_center[1]) ** 2
            q_laser = 2 * eta * P / (np.pi * rb ** 2) * np.exp(-2 * d2 / rb ** 2) * switch
            q_conv = h * (T0 - old_T[0])
            q_rad = SB_constant * emissivity * (T0 ** 4 - old_T[0] ** 4)
            q = q_conv + q_rad + q_laser
            return -np.array([q])
 
        def thermal_neumann_walls(u, point, old_T):
            # q is the heat flux into the domain.
            q_conv = h * (T0 - old_T[0])
            q_rad = SB_constant * emissivity * (T0 ** 4 - old_T[0] ** 4)
            q = q_conv + q_rad
            return -np.array([q])

        return [thermal_neumann_top, thermal_neumann_walls]

    def set_params(self, params):
        # Override base class method.
        sol_T_old, laser_center, switch = params

        # ?????
        sol_T_old_top = self.fes[0].convert_from_dof_to_face_quad(sol_T_old, self.boundary_inds_list[0])
        sol_T_old_walls = self.fes[0].convert_from_dof_to_face_quad(sol_T_old, self.boundary_inds_list[1])

        # (num_selected_faces, num_face_quads, dim)
        laser_center_quad = laser_center[None, None, :] * np.ones((len(self.boundary_inds_list[0]), self.fes[0].num_face_quads))[:, :, None]
        # (num_selected_faces, num_face_quads)
        switch_quad = switch * np.ones((len(self.boundary_inds_list[0]), self.fes[0].num_face_quads))

        self.internal_vars_surfaces = [[sol_T_old_top, laser_center_quad, switch_quad], [sol_T_old_walls]]
        self.internal_vars = [self.fes[0].convert_from_dof_to_quad(sol_T_old)]

In [25]:
# Define material properties. 
# We generally assume Inconel 625 material is used. 
# SI units are used throughout this example.
Cp = 588. # specific heat capacity (J/kg·K)
rho = 8440. # material density (kg/m^3)
k = 15. # thermal conductivity (W/m·K)
Tl = 1623 # liquidus temperature (K)
h = 100. # heat convection coefficien (W/m^2·K)
eta = 0.25 # absorption rate 0.37
SB_constant = 5.67e-8 # Stefan-Boltzmann constant (kg·s^-3·K^-4)
emissivity = 0.3 # emissivity
T0 = 300. # ambient temperature (K)
POWDER = 0 # powder flag
LIQUID = 1 # liquid flag
SOLID = 2 # solid flag

# Define laser properties.
vel = 0.5 # laser scanning velocity (m/s)
rb = 1e-4 # laser beam size (m) 0.05e-3
P = 150. # laser power (W)

In [ ]:
# Specify mesh-related information. 
# We use first-order hexahedron element for both T_crt and u_crt.
ele_type = 'HEX8'
cell_type = get_meshio_cell_type(ele_type)
Nx, Ny, Nz = 1000, 500, 250  # 10 um per element 50, 20, 5
Lx, Ly, Lz = 1e-2, 0.5e-2, 0.25e-2  # domain size unit??? (m) 0.5e-3, 0.2e-3, 0.05e-3
meshio_mesh = box_mesh_gmsh(Nx, Ny, Nz, Lx, Ly, Lz, data_dir)
mesh = Mesh(meshio_mesh.points, meshio_mesh.cells_dict[cell_type])

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Extruded)
Info    : [ 10%] Meshing curve 2 (Extruded)
Info    : [ 20%] Meshing curve 3 (Extruded)
Info    : [ 30%] Meshing curve 4 (Extruded)
Info    : [ 40%] Meshing curve 7 (Extruded)
Info    : [ 50%] Meshing curve 8 (Extruded)
Info    : [ 50%] Meshing curve 9 (Extruded)
Info    : [ 60%] Meshing curve 10 (Extruded)
Info    : [ 70%] Meshing curve 12 (Extruded)
Info    : [ 80%] Meshing curve 13 (Extruded)
Info    : [ 90%] Meshing curve 17 (Extruded)
Info    : [100%] Meshing curve 21 (Extruded)
Info    : Done meshing 1D (Wall 0.00432952s, CPU 0.005982s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 5 (Extruded)
Info    : [ 20%] Meshing surface 14 (Extruded)
Info    : [ 40%] Meshing surface 18 (Extruded)
Info    : [ 50%] Meshing surface 22 (Extruded)
Info    : [ 70%] Meshing surface 26 (Extruded)
Info    : [ 90%] Meshing surface 27 (Extruded)
Info    : Done meshing 2D (Wall 5.28683s, CPU 5.19353s)
Info    : Meshing 3D..

In [16]:
# Define boundary locations.
def top(point):
    return np.isclose(point[2], Lz, atol=1e-5)

def bottom(point):
    return np.isclose(point[2], 0., atol=1e-5)

def walls(point):
    left = np.isclose(point[0], 0., atol=1e-5)
    right = np.isclose(point[0], Lx, atol=1e-5)
    front = np.isclose(point[1], 0., atol=1e-5)
    back = np.isclose(point[1], Ly, atol=1e-5)
    return left | right | front | back

# Specify boundary conditions and problem definitions.
# Dirichlet BC values for thermal problem
def thermal_dirichlet_bottom(point):
    return T0

# Define thermal problem
dirichlet_bc_info_T = [[bottom], [0], [thermal_dirichlet_bottom]]
location_fns = [top, walls]

sol_T_old = T0 * np.ones((len(mesh.points), 1))
sol_T_old_for_u = np.array(sol_T_old)
problem_T = Thermal(mesh, vec=1, dim=3, dirichlet_bc_info=dirichlet_bc_info_T, location_fns=location_fns)

[01-07 20:47:17][DEBUG] jax_fem: Computing shape function values, gradients, etc.
[01-07 20:47:17][DEBUG] jax_fem: ele_type = HEX8, quad_points.shape = (num_quads, dim) = (8, 3)
[01-07 20:47:17][DEBUG] jax_fem: face_quad_points.shape = (num_faces, num_face_quads, dim) = (6, 4, 3)
[01-07 20:47:17][DEBUG] jax_fem: Done pre-computations, took 0.6488907337188721 [s]
[01-07 20:47:17][INFO] jax_fem: Solving a problem with 5000 cells, 6426x1 = 6426 dofs.


In [17]:
# Do some cleaning work.
files = glob.glob(os.path.join(vtk_dir, f'*'))
for f in files:
    os.remove(f)

In [18]:
# Save initial solution to local folder.
vtk_path = os.path.join(vtk_dir, f"u_{0:05d}.vtu")
save_sol(problem_T.fes[0], sol_T_old, vtk_path)

In [19]:
# Start the major loop of time iteration.
dt = 2 * 1e-6  # time increment
laser_on_t = 0.5 * Lx / vel  # time duration for scanning over half the length
simulation_t = 2 * laser_on_t  # total simulation time
ts = np.arange(0., simulation_t, dt)  # time steps

for i in range(len(ts[1:, 200])):
    laser_center = np.array([Lx * 0.25 + vel * ts[i + 1], Ly / 2., Lz])  # laser center in the middle of the powder
    switch = np.where(ts[i + 1] < laser_on_t, 1., 0.) # Turn off the laser after some time
    print(f"\nStep {i + 1}, total step = {len(ts[1:])}, laser_x = {laser_center[0]}, Lx = {Lx}, laser ON = {ts[i + 1] < laser_on_t}")

    # Set parameter and solve for T
    problem_T.set_params([sol_T_old, laser_center, switch])
    sol_T_new_list = solver(problem_T)
    sol_T_new = sol_T_new_list[0]
    
    vtk_path = os.path.join(vtk_dir, f"u_{i + 1:05d}.vtu")
    save_sol(problem_T.fes[0], sol_T_new, vtk_path)
    
    # Update T solution
    sol_T_old = sol_T_new


Step 1, total step = 500, laser_x = 0.000126, Lx = 0.0005, laser ON = True


[01-07 20:47:38][DEBUG] jax_fem: Calling the row elimination solver for imposing Dirichlet B.C.
[01-07 20:47:38][DEBUG] jax_fem: Start timing
[01-07 20:47:38][DEBUG] jax_fem: Computing cell Jacobian and cell residual...
  val = Traced<ShapedArray(float64[8,1])>with<JVPTrace(level=3/1)> with
    primal = Traced<ShapedArray(float64[8,1])>with<BatchTrace(level=1/1)> with
      val = Traced<ShapedArray(float64[250,8,1])>with<DynamicJaxprTrace(level=0/1)>
      batch_dim = 0
    tangent = Traced<ShapedArray(float64[8,1])>with<BatchTrace(level=2/1)> with
      val = Traced<ShapedArray(float64[8,8,1])>with<DynamicJaxprTrace(level=0/1)>
      batch_dim = 0
  batch_dim = 0 shape (1,)
  val = Traced<ShapedArray(float64[8,3])>with<BatchTrace(level=1/1)> with
    val = Traced<ShapedArray(float64[250,8,3])>with<DynamicJaxprTrace(level=0/1)>
    batch_dim = 0
  batch_dim = 0 x (3,)
  val = Traced<ShapedArray(float64[8,1,3])>with<JVPTrace(level=3/1)> with
    primal = Traced<ShapedArray(float64[8,1,3


Step 2, total step = 500, laser_x = 0.000127, Lx = 0.0005, laser ON = True


[01-07 20:47:41][DEBUG] jax_fem: JAX Solver - Finshed solving, res = 8.100352857262275e-07
[01-07 20:47:41][DEBUG] jax_fem: Computing cell Jacobian and cell residual...
[01-07 20:47:41][DEBUG] jax_fem: Function split_and_compute_cell took 0.0367 seconds
[01-07 20:47:41][DEBUG] jax_fem: Creating sparse matrix with scipy...
[01-07 20:47:41][DEBUG] jax_fem: res l_2 = 8.100352863573473e-07
[01-07 20:47:41][INFO] jax_fem: Solve took 0.5833656787872314 [s]
[01-07 20:47:41][DEBUG] jax_fem: max of dofs = 1919.8718999712148
[01-07 20:47:41][DEBUG] jax_fem: min of dofs = 206.55633090457835
[01-07 20:47:41][DEBUG] jax_fem: Calling the row elimination solver for imposing Dirichlet B.C.
[01-07 20:47:41][DEBUG] jax_fem: Start timing
[01-07 20:47:41][DEBUG] jax_fem: Computing cell Jacobian and cell residual...
[01-07 20:47:41][DEBUG] jax_fem: Function split_and_compute_cell took 0.0353 seconds
[01-07 20:47:41][DEBUG] jax_fem: Creating sparse matrix with scipy...
[01-07 20:47:41][DEBUG] jax_fem: Befor


Step 3, total step = 500, laser_x = 0.000128, Lx = 0.0005, laser ON = True


...

[01-07 20:53:04][DEBUG] jax_fem: JAX Solver - Finshed solving, res = 1.2782249226214519e-07
[01-07 20:53:04][DEBUG] jax_fem: Computing cell Jacobian and cell residual...
[01-07 20:53:04][DEBUG] jax_fem: Function split_and_compute_cell took 0.0316 seconds
[01-07 20:53:04][DEBUG] jax_fem: Creating sparse matrix with scipy...
[01-07 20:53:04][DEBUG] jax_fem: res l_2 = 1.2782249254687407e-07
[01-07 20:53:04][INFO] jax_fem: Solve took 0.5455913543701172 [s]
[01-07 20:53:04][DEBUG] jax_fem: max of dofs = 691.7245022641083
[01-07 20:53:04][DEBUG] jax_fem: min of dofs = 300.0
[01-07 20:53:04][DEBUG] jax_fem: Calling the row elimination solver for imposing Dirichlet B.C.
[01-07 20:53:04][DEBUG] jax_fem: Start timing
[01-07 20:53:04][DEBUG] jax_fem: Computing cell Jacobian and cell residual...
[01-07 20:53:04][DEBUG] jax_fem: Function split_and_compute_cell took 0.0363 seconds
[01-07 20:53:04][DEBUG] jax_fem: Creating sparse matrix with scipy...
[01-07 20:53:04][DEBUG] jax_fem: Before, res l_2 =


Step 499, total step = 500, laser_x = 0.000624, Lx = 0.0005, laser ON = False


[01-07 20:53:05][DEBUG] jax_fem: JAX Solver - Finshed solving, res = 1.2588862532396248e-07
[01-07 20:53:05][DEBUG] jax_fem: Computing cell Jacobian and cell residual...
[01-07 20:53:05][DEBUG] jax_fem: Function split_and_compute_cell took 0.0412 seconds
[01-07 20:53:05][DEBUG] jax_fem: Creating sparse matrix with scipy...
[01-07 20:53:05][DEBUG] jax_fem: res l_2 = 1.2588862518717386e-07
[01-07 20:53:05][INFO] jax_fem: Solve took 0.5497903823852539 [s]
[01-07 20:53:05][DEBUG] jax_fem: max of dofs = 687.2446811486556
[01-07 20:53:05][DEBUG] jax_fem: min of dofs = 300.0
[01-07 20:53:05][DEBUG] jax_fem: Calling the row elimination solver for imposing Dirichlet B.C.
[01-07 20:53:05][DEBUG] jax_fem: Start timing
[01-07 20:53:05][DEBUG] jax_fem: Computing cell Jacobian and cell residual...
[01-07 20:53:05][DEBUG] jax_fem: Function split_and_compute_cell took 0.0361 seconds
[01-07 20:53:05][DEBUG] jax_fem: Creating sparse matrix with scipy...
[01-07 20:53:05][DEBUG] jax_fem: Before, res l_2 =


Step 500, total step = 500, laser_x = 0.000625, Lx = 0.0005, laser ON = False


[01-07 20:53:05][DEBUG] jax_fem: JAX Solver - Finshed solving, res = 1.2400750315939885e-07
[01-07 20:53:05][DEBUG] jax_fem: Computing cell Jacobian and cell residual...
[01-07 20:53:06][DEBUG] jax_fem: Function split_and_compute_cell took 0.0383 seconds
[01-07 20:53:06][DEBUG] jax_fem: Creating sparse matrix with scipy...
[01-07 20:53:06][DEBUG] jax_fem: res l_2 = 1.2400750328968906e-07
[01-07 20:53:06][INFO] jax_fem: Solve took 0.7446138858795166 [s]
[01-07 20:53:06][DEBUG] jax_fem: max of dofs = 682.8191292780477
[01-07 20:53:06][DEBUG] jax_fem: min of dofs = 300.0


## Test with larger thickness

Already tested:
- Larger thickness -> possible reason for simulated extreme temperature
- Realistic laser and material configuration
- Mesh size -> 10 um or 40 um
- Boundary condition -> 5 Neumann + 1 Dirichlet boundaries

Other reasons:
- Time increment

In [ ]:
# Import some useful modules.
import jax
import jax.numpy as np
import os
import glob

# Import JAX-FEM specific modules.
from jax_fem.generate_mesh import box_mesh_gmsh, Mesh, get_meshio_cell_type
from jax_fem.solver import solver
from jax_fem.problem import Problem
from jax_fem.utils import save_sol

# If you have multiple GPUs, set the one to use.
device = ' cpu'

if device == 'gpu':
    os.environ["CUDA_VISIBLE_DEVICES"] = "2"

# Define some useful directory paths.
# crt_file_path = os.path.dirname(__file__)
crt_file_path = os.getcwd()
data_dir = os.path.join(crt_file_path, '05a_data')
vtk_dir = os.path.join(data_dir, 'vtk')

# Define the thermal problem. 
# We solve the following equation (weak form of FEM):
# (rho*Cp/dt*(T_crt-T_old), Q) * dx + (k*T_crt_grad, Q_grad) * dx - (heat_flux, Q) * ds = 0
# where T_crt is the trial function, and Q is the test function.
class Thermal(Problem):
    # The function 'get_tensor_map' is responsible for the term (k*T_crt_grad, Q_grad) * dx. 
    # The function 'get_mass_map' is responsible for the term (rho*Cp/dt*(T_crt-T_old), Q) * dx. 
    # The function 'set_params' makes sure that the Neumann boundary conditions use the most 
    # updated T_old and laser information, i.e., positional information like laser_center (x_l, y_l) and 
    # 'switch' controlling ON/OFF of the laser.
    def get_tensor_map(self):
        def fn(u_grad, T_old):
            return k * u_grad
        return fn
 
    def get_mass_map(self):
        def T_map(T, x, T_old):
            return rho * Cp * (T - T_old) / dt
        return T_map

    def get_surface_maps(self):
        # Neumann BC values for thermal problem
        def thermal_neumann_top(u, point, old_T, laser_center, switch):
            # q is the heat flux into the domain
            d2 = (point[0] - laser_center[0]) ** 2 + (point[1] - laser_center[1]) ** 2
            q_laser = 2 * eta * P / (np.pi * rb ** 2) * np.exp(-2 * d2 / rb ** 2) * switch
            q_conv = h * (T0 - old_T[0])
            q_rad = SB_constant * emissivity * (T0 ** 4 - old_T[0] ** 4)
            q = q_conv + q_rad + q_laser
            return -np.array([q])
 
        def thermal_neumann_walls(u, point, old_T):
            # q is the heat flux into the domain.
            q_conv = h * (T0 - old_T[0])
            q_rad = SB_constant * emissivity * (T0 ** 4 - old_T[0] ** 4)
            q = q_conv + q_rad
            return -np.array([q])

        return [thermal_neumann_top, thermal_neumann_walls]

    def set_params(self, params):
        # Override base class method.
        sol_T_old, laser_center, switch = params

        # ?????
        sol_T_old_top = self.fes[0].convert_from_dof_to_face_quad(sol_T_old, self.boundary_inds_list[0])
        sol_T_old_walls = self.fes[0].convert_from_dof_to_face_quad(sol_T_old, self.boundary_inds_list[1])

        # (num_selected_faces, num_face_quads, dim)
        laser_center_quad = laser_center[None, None, :] * np.ones((len(self.boundary_inds_list[0]), self.fes[0].num_face_quads))[:, :, None]
        # (num_selected_faces, num_face_quads)
        switch_quad = switch * np.ones((len(self.boundary_inds_list[0]), self.fes[0].num_face_quads))

        self.internal_vars_surfaces = [[sol_T_old_top, laser_center_quad, switch_quad], [sol_T_old_walls]]
        self.internal_vars = [self.fes[0].convert_from_dof_to_quad(sol_T_old)]

# Define material properties. 
# We generally assume Inconel 625 material is used. 
# SI units are used throughout this example.
Cp = 670. # heat capacity (J/kg·K)
rho = 8440. # material density (kg/m^3)
k = 25.2 # thermal conductivity (W/m·K)
Tl = 1623 # liquidus temperature (K)
h = 100. # heat convection coefficien (W/m^2·K)
eta = 0.25 # absorption rate
SB_constant = 5.67e-8 # Stefan-Boltzmann constant (kg·s^-3·K^-4)
emissivity = 0.3 # emissivity
T0 = 293. # ambient temperature (K)
POWDER = 0 # powder flag
LIQUID = 1 # liquid flag
SOLID = 2 # solid flag

# Define laser properties.
vel = 0.875 # laser scanning velocity (m/s)
rb = 0.1e-3 # laser beam size (m)
P = 169. # laser power (W)

# Specify mesh-related information. 
# We use first-order hexahedron element for both T_crt and u_crt.
ele_type = 'HEX8'
cell_type = get_meshio_cell_type(ele_type)
Nx, Ny, Nz = 50, 50, 25  # 10 um per element
Lx, Ly, Lz = 0.5e-3, 0.5e-3, 0.25e-3 # domain size (m)
meshio_mesh = box_mesh_gmsh(Nx, Ny, Nz, Lx, Ly, Lz, data_dir)
mesh = Mesh(meshio_mesh.points, meshio_mesh.cells_dict[cell_type])

# Define boundary locations.
def top(point):
    return np.isclose(point[2], Lz, atol=1e-5)

def bottom(point):
    return np.isclose(point[2], 0., atol=1e-5)

def walls(point):
    left = np.isclose(point[0], 0., atol=1e-5)
    right = np.isclose(point[0], Lx, atol=1e-5)
    front = np.isclose(point[1], 0., atol=1e-5)
    back = np.isclose(point[1], Ly, atol=1e-5)
    return left | right | front | back

# Specify boundary conditions and problem definitions.
# Dirichlet BC values for thermal problem
def thermal_dirichlet_bottom(point):
    return T0

# Define thermal problem
dirichlet_bc_info_T = [[bottom], [0], [thermal_dirichlet_bottom]]
location_fns = [top, walls]

sol_T_old = T0 * np.ones((len(mesh.points), 1))
sol_T_old_for_u = np.array(sol_T_old)
problem_T = Thermal(mesh, vec=1, dim=3, dirichlet_bc_info=dirichlet_bc_info_T, location_fns=location_fns)

# Do some cleaning work.
files = glob.glob(os.path.join(vtk_dir, f'*'))
for f in files:
    os.remove(f)

# Save initial solution to local folder.
vtk_path = os.path.join(vtk_dir, f"u_{0:05d}.vtu")
save_sol(problem_T.fes[0], sol_T_old, vtk_path)

# Start the major loop of time iteration.
dt = 2 * 1e-6  # time increment
laser_on_t = 0.5 * Lx / vel  # time duration for scanning over half the length
simulation_t = 2 * laser_on_t  # total simulation time
ts = np.arange(0., simulation_t, dt)  # time steps

for i in range(len(ts[1:])):
    laser_center = np.array([Lx * 0.25 + vel * ts[i + 1], Ly / 2., Lz])  # laser center in the middle of the powder
    switch = np.where(ts[i + 1] < laser_on_t, 1., 0.) # Turn off the laser after some time
    print(f"\nStep {i + 1}, total step = {len(ts[1:])}, laser_x = {laser_center[0]}, Lx = {Lx}, laser ON = {ts[i + 1] < laser_on_t}")

    # Set parameter and solve for T
    problem_T.set_params([sol_T_old, laser_center, switch])
    sol_T_new_list = solver(problem_T)
    sol_T_new = sol_T_new_list[0]
    
    vtk_path = os.path.join(vtk_dir, f"u_{i + 1:05d}.vtu")
    save_sol(problem_T.fes[0], sol_T_new, vtk_path)
    
    # Update T solution
    sol_T_old = sol_T_new


Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Extruded)
Info    : [ 10%] Meshing curve 2 (Extruded)
Info    : [ 20%] Meshing curve 3 (Extruded)
Info    : [ 30%] Meshing curve 4 (Extruded)
Info    : [ 40%] Meshing curve 7 (Extruded)
Info    : [ 50%] Meshing curve 8 (Extruded)
Info    : [ 50%] Meshing curve 9 (Extruded)
Info    : [ 60%] Meshing curve 10 (Extruded)
Info    : [ 70%] Meshing curve 12 (Extruded)
Info    : [ 80%] Meshing curve 13 (Extruded)
Info    : [ 90%] Meshing curve 17 (Extruded)
Info    : [100%] Meshing curve 21 (Extruded)
Info    : Done meshing 1D (Wall 0.0100893s, CPU 0.010012s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 5 (Extruded)
Info    : [ 20%] Meshing surface 14 (Extruded)
Info    : [ 40%] Meshing surface 18 (Extruded)
Info    : [ 50%] Meshing surface 22 (Extruded)
Info    : [ 70%] Meshing surface 26 (Extruded)
Info    : [ 90%] Meshing surface 27 (Extruded)
Info    : Done meshing 2D (Wall 1.07998s, CPU 1.01822s)
Info    : Meshing 3D...

[01-08 10:11:52][DEBUG] jax_fem: Computing shape function values, gradients, etc.
[01-08 10:11:52][DEBUG] jax_fem: ele_type = HEX8, quad_points.shape = (num_quads, dim) = (8, 3)
[01-08 10:11:52][DEBUG] jax_fem: face_quad_points.shape = (num_faces, num_face_quads, dim) = (6, 4, 3)


## Runable experiement 1

In [ ]:
# Import some useful modules.
import jax
import jax.numpy as np
import os
import glob

# Import JAX-FEM specific modules.
from jax_fem.generate_mesh import box_mesh_gmsh, Mesh, get_meshio_cell_type
from jax_fem.solver import solver
from jax_fem.problem import Problem
from jax_fem.utils import save_sol

# If you have multiple GPUs, set the one to use.
device = ' cpu'

if device == 'gpu':
    os.environ["CUDA_VISIBLE_DEVICES"] = "2"

# Define some useful directory paths.
# crt_file_path = os.path.dirname(__file__)
crt_file_path = os.getcwd()
data_dir = os.path.join(crt_file_path, '05a_data')
vtk_dir = os.path.join(data_dir, 'vtk')

# Define the thermal problem. 
# We solve the following equation (weak form of FEM):
# (rho*Cp/dt*(T_crt-T_old), Q) * dx + (k*T_crt_grad, Q_grad) * dx - (heat_flux, Q) * ds = 0
# where T_crt is the trial function, and Q is the test function.
class Thermal(Problem):
    # The function 'get_tensor_map' is responsible for the term (k*T_crt_grad, Q_grad) * dx. 
    # The function 'get_mass_map' is responsible for the term (rho*Cp/dt*(T_crt-T_old), Q) * dx. 
    # The function 'set_params' makes sure that the Neumann boundary conditions use the most 
    # updated T_old and laser information, i.e., positional information like laser_center (x_l, y_l) and 
    # 'switch' controlling ON/OFF of the laser.
    def get_tensor_map(self):
        def fn(u_grad, T_old):
            return k * u_grad
        return fn
 
    def get_mass_map(self):
        def T_map(T, x, T_old):
            return rho * Cp * (T - T_old) / dt
        return T_map

    def get_surface_maps(self):
        # Neumann BC values for thermal problem
        def thermal_neumann_top(u, point, old_T, laser_center, switch):
            # q is the heat flux into the domain
            d2 = (point[0] - laser_center[0]) ** 2 + (point[1] - laser_center[1]) ** 2
            q_laser = 2 * eta * P / (np.pi * rb ** 2) * np.exp(-2 * d2 / rb ** 2) * switch
            q_conv = h * (T0 - old_T[0])
            q_rad = SB_constant * emissivity * (T0 ** 4 - old_T[0] ** 4)
            q = q_conv + q_rad + q_laser
            return -np.array([q])
 
        def thermal_neumann_walls(u, point, old_T):
            # q is the heat flux into the domain.
            q_conv = h * (T0 - old_T[0])
            q_rad = SB_constant * emissivity * (T0 ** 4 - old_T[0] ** 4)
            q = q_conv + q_rad
            return -np.array([q])

        return [thermal_neumann_top, thermal_neumann_walls]

    def set_params(self, params):
        # Override base class method.
        sol_T_old, laser_center, switch = params

        # ?????
        sol_T_old_top = self.fes[0].convert_from_dof_to_face_quad(sol_T_old, self.boundary_inds_list[0])
        sol_T_old_walls = self.fes[0].convert_from_dof_to_face_quad(sol_T_old, self.boundary_inds_list[1])

        # (num_selected_faces, num_face_quads, dim)
        laser_center_quad = laser_center[None, None, :] * np.ones((len(self.boundary_inds_list[0]), self.fes[0].num_face_quads))[:, :, None]
        # (num_selected_faces, num_face_quads)
        switch_quad = switch * np.ones((len(self.boundary_inds_list[0]), self.fes[0].num_face_quads))

        self.internal_vars_surfaces = [[sol_T_old_top, laser_center_quad, switch_quad], [sol_T_old_walls]]
        self.internal_vars = [self.fes[0].convert_from_dof_to_quad(sol_T_old)]

# Define material properties. 
# We generally assume Inconel 625 material is used. 
# SI units are used throughout this example.
Cp = 670. # heat capacity (J/kg·K)
rho = 8440. # material density (kg/m^3)
k = 25.2 # thermal conductivity (W/m·K)
Tl = 1623 # liquidus temperature (K)
h = 100. # heat convection coefficien (W/m^2·K)
eta = 0.25 # absorption rate
SB_constant = 5.67e-8 # Stefan-Boltzmann constant (kg·s^-3·K^-4)
emissivity = 0.3 # emissivity
T0 = 293. # ambient temperature (K)
POWDER = 0 # powder flag
LIQUID = 1 # liquid flag
SOLID = 2 # solid flag

# Define laser properties.
vel = 0.875 # laser scanning velocity (m/s)
rb = 0.1e-3 # laser beam size (m)
P = 169. # laser power (W)

# Specify mesh-related information. 
# We use first-order hexahedron element for both T_crt and u_crt.
ele_type = 'HEX8'
cell_type = get_meshio_cell_type(ele_type)
Nx, Ny, Nz = 50, 50, 25  # 10*4 um per element
Lx, Ly, Lz = 2e-3, 2e-3, 1e-3 # domain size (m)
meshio_mesh = box_mesh_gmsh(Nx, Ny, Nz, Lx, Ly, Lz, data_dir)
mesh = Mesh(meshio_mesh.points, meshio_mesh.cells_dict[cell_type])

# Define boundary locations.
def top(point):
    return np.isclose(point[2], Lz, atol=1e-5)

def bottom(point):
    return np.isclose(point[2], 0., atol=1e-5)

def walls(point):
    left = np.isclose(point[0], 0., atol=1e-5)
    right = np.isclose(point[0], Lx, atol=1e-5)
    front = np.isclose(point[1], 0., atol=1e-5)
    back = np.isclose(point[1], Ly, atol=1e-5)
    return left | right | front | back

# Specify boundary conditions and problem definitions.
# Dirichlet BC values for thermal problem
def thermal_dirichlet_bottom(point):
    return T0

# Define thermal problem
dirichlet_bc_info_T = [[bottom], [0], [thermal_dirichlet_bottom]]
location_fns = [top, walls]

sol_T_old = T0 * np.ones((len(mesh.points), 1))
sol_T_old_for_u = np.array(sol_T_old)
problem_T = Thermal(mesh, vec=1, dim=3, dirichlet_bc_info=dirichlet_bc_info_T, location_fns=location_fns)

# Do some cleaning work.
files = glob.glob(os.path.join(vtk_dir, f'*'))
for f in files:
    os.remove(f)

# Save initial solution to local folder.
vtk_path = os.path.join(vtk_dir, f"u_{0:05d}.vtu")
save_sol(problem_T.fes[0], sol_T_old, vtk_path)

# Start the major loop of time iteration.
dt = 10 * 1e-6  # time increment ???
laser_on_t = 0.5 * Lx / vel  # time duration for scanning over half the length
simulation_t = 2 * laser_on_t  # total simulation time
ts = np.arange(0., simulation_t, dt)  # time steps

for i in range(len(ts[1:100])):
    laser_center = np.array([Lx * 0.25 + vel * ts[i + 1], Ly / 2., Lz])  # laser center in the middle of the powder
    switch = np.where(ts[i + 1] < laser_on_t, 1., 0.) # Turn off the laser after some time
    print(f"\nStep {i + 1}, total step = {len(ts[1:])}, laser_x = {laser_center[0]}, Lx = {Lx}, laser ON = {ts[i + 1] < laser_on_t}")

    # Set parameter and solve for T
    problem_T.set_params([sol_T_old, laser_center, switch])
    sol_T_new_list = solver(problem_T)
    sol_T_new = sol_T_new_list[0]
    
    vtk_path = os.path.join(vtk_dir, f"u_{i + 1:05d}.vtu")
    save_sol(problem_T.fes[0], sol_T_new, vtk_path)
    
    # Update T solution
    sol_T_old = sol_T_new

## Runable experiment 2

In [ ]:
# Import some useful modules.
import jax
import jax.numpy as np
import os
import glob

# Import JAX-FEM specific modules.
from jax_fem.generate_mesh import box_mesh_gmsh, Mesh, get_meshio_cell_type
from jax_fem.solver import solver
from jax_fem.problem import Problem
from jax_fem.utils import save_sol

# If you have multiple GPUs, set the one to use.
device = ' cpu'

if device == 'gpu':
    os.environ["CUDA_VISIBLE_DEVICES"] = "2"

# Define some useful directory paths.
# crt_file_path = os.path.dirname(__file__)
crt_file_path = os.getcwd()
data_dir = os.path.join(crt_file_path, '05a_data')
vtk_dir = os.path.join(data_dir, 'vtk')

# Define the thermal problem. 
# We solve the following equation (weak form of FEM):
# (rho*Cp/dt*(T_crt-T_old), Q) * dx + (k*T_crt_grad, Q_grad) * dx - (heat_flux, Q) * ds = 0
# where T_crt is the trial function, and Q is the test function.
class Thermal(Problem):
    # The function 'get_tensor_map' is responsible for the term (k*T_crt_grad, Q_grad) * dx. 
    # The function 'get_mass_map' is responsible for the term (rho*Cp/dt*(T_crt-T_old), Q) * dx. 
    # The function 'set_params' makes sure that the Neumann boundary conditions use the most 
    # updated T_old and laser information, i.e., positional information like laser_center (x_l, y_l) and 
    # 'switch' controlling ON/OFF of the laser.
    def get_tensor_map(self):
        def fn(u_grad, T_old):
            return k * u_grad
        return fn
 
    def get_mass_map(self):
        def T_map(T, x, T_old):
            return rho * Cp * (T - T_old) / dt
        return T_map

    def get_surface_maps(self):
        # Neumann BC values for thermal problem
        def thermal_neumann_top(u, point, old_T, laser_center, switch):
            # q is the heat flux into the domain
            d2 = (point[0] - laser_center[0]) ** 2 + (point[1] - laser_center[1]) ** 2
            q_laser = 2 * eta * P / (np.pi * rb ** 2) * np.exp(-2 * d2 / rb ** 2) * switch
            q_conv = h * (T0 - old_T[0])
            q_rad = SB_constant * emissivity * (T0 ** 4 - old_T[0] ** 4)
            q = q_conv + q_rad + q_laser
            return -np.array([q])
 
        def thermal_neumann_walls(u, point, old_T):
            # q is the heat flux into the domain.
            q_conv = h * (T0 - old_T[0])
            q_rad = SB_constant * emissivity * (T0 ** 4 - old_T[0] ** 4)
            q = q_conv + q_rad
            return -np.array([q])

        return [thermal_neumann_top, thermal_neumann_walls]

    def set_params(self, params):
        # Override base class method.
        sol_T_old, laser_center, switch = params

        # ?????
        sol_T_old_top = self.fes[0].convert_from_dof_to_face_quad(sol_T_old, self.boundary_inds_list[0])
        sol_T_old_walls = self.fes[0].convert_from_dof_to_face_quad(sol_T_old, self.boundary_inds_list[1])

        # (num_selected_faces, num_face_quads, dim)
        laser_center_quad = laser_center[None, None, :] * np.ones((len(self.boundary_inds_list[0]), self.fes[0].num_face_quads))[:, :, None]
        # (num_selected_faces, num_face_quads)
        switch_quad = switch * np.ones((len(self.boundary_inds_list[0]), self.fes[0].num_face_quads))

        self.internal_vars_surfaces = [[sol_T_old_top, laser_center_quad, switch_quad], [sol_T_old_walls]]
        self.internal_vars = [self.fes[0].convert_from_dof_to_quad(sol_T_old)]

# Define material properties. 
# We generally assume Inconel 625 material is used. 
# SI units are used throughout this example.
Cp = 670. # heat capacity (J/kg·K)
rho = 8440. # material density (kg/m^3)
k = 25.2 # thermal conductivity (W/m·K)
Tl = 1623 # liquidus temperature (K)
h = 100. # heat convection coefficien (W/m^2·K)
eta = 0.25 # absorption rate
SB_constant = 5.67e-8 # Stefan-Boltzmann constant (kg·s^-3·K^-4)
emissivity = 0.3 # emissivity
T0 = 293. # ambient temperature (K)
POWDER = 0 # powder flag
LIQUID = 1 # liquid flag
SOLID = 2 # solid flag

# Define laser properties.
vel = 0.875 # laser scanning velocity (m/s)
rb = 0.1e-3 # laser beam size (m)
P = 169. # laser power (W)

# Specify mesh-related information. 
# We use first-order hexahedron element for both T_crt and u_crt.
ele_type = 'HEX8'
cell_type = get_meshio_cell_type(ele_type)
Nx, Ny, Nz = 100, 100, 50  # 10 um per element
Lx, Ly, Lz = 2e-3, 2e-3, 1e-3 # domain size (m)
meshio_mesh = box_mesh_gmsh(Nx, Ny, Nz, Lx, Ly, Lz, data_dir)
mesh = Mesh(meshio_mesh.points, meshio_mesh.cells_dict[cell_type])

# Define boundary locations.
def top(point):
    return np.isclose(point[2], Lz, atol=1e-5)

def bottom(point):
    return np.isclose(point[2], 0., atol=1e-5)

def walls(point):
    left = np.isclose(point[0], 0., atol=1e-5)
    right = np.isclose(point[0], Lx, atol=1e-5)
    front = np.isclose(point[1], 0., atol=1e-5)
    back = np.isclose(point[1], Ly, atol=1e-5)
    return left | right | front | back

# Specify boundary conditions and problem definitions.
# Dirichlet BC values for thermal problem
def thermal_dirichlet_bottom(point):
    return T0

# Define thermal problem
dirichlet_bc_info_T = [[bottom], [0], [thermal_dirichlet_bottom]]
location_fns = [top, walls]

sol_T_old = T0 * np.ones((len(mesh.points), 1))
sol_T_old_for_u = np.array(sol_T_old)
problem_T = Thermal(mesh, vec=1, dim=3, dirichlet_bc_info=dirichlet_bc_info_T, location_fns=location_fns)

# Do some cleaning work.
files = glob.glob(os.path.join(vtk_dir, f'*'))
for f in files:
    os.remove(f)

# Save initial solution to local folder.
vtk_path = os.path.join(vtk_dir, f"u_{0:05d}.vtu")
save_sol(problem_T.fes[0], sol_T_old, vtk_path)

# Start the major loop of time iteration.
dt = 10 * 1e-6  # time increment ???
laser_on_t = 0.5 * Lx / vel  # time duration for scanning over half the length
simulation_t = 2 * laser_on_t  # total simulation time
ts = np.arange(0., simulation_t, dt)  # time steps

for i in range(len(ts[1:100])):
    laser_center = np.array([Lx * 0.25 + vel * ts[i + 1], Ly / 2., Lz])  # laser center in the middle of the powder
    switch = np.where(ts[i + 1] < laser_on_t, 1., 0.) # Turn off the laser after some time
    print(f"\nStep {i + 1}, total step = {len(ts[1:])}, laser_x = {laser_center[0]}, Lx = {Lx}, laser ON = {ts[i + 1] < laser_on_t}")

    # Set parameter and solve for T
    problem_T.set_params([sol_T_old, laser_center, switch])
    sol_T_new_list = solver(problem_T)
    sol_T_new = sol_T_new_list[0]
    
    vtk_path = os.path.join(vtk_dir, f"u_{i + 1:05d}.vtu")
    save_sol(problem_T.fes[0], sol_T_new, vtk_path)
    
    # Update T solution
    sol_T_old = sol_T_new

# Read and extract data from .vtu files